In [49]:
#2 Importing Relevant Libraries

import json
import string
import random 

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
#3 Loading the Dataset: intents.json

data_file = open('/content/intentsfr.json').read()
data = json.loads(data_file)

data

{'intents': [{'context': '',
   'patterns': ['Salut', 'Hey', 'Bonjour', 'Hello', 'salut'],
   'responses': ['Hey!', 'Salut à vous!', 'Comment-allez vous', 'Salut-salut'],
   'tag': 'hello'},
  {'context': [''],
   'patterns': [],
   'responses': ['Je suis désolé mais je ne comprend pas',
    "Donnez moi plus d'informations",
    'Je ne suis pas sur de comprendre'],
   'tag': 'noanswer'},
  {'context': '',
   'patterns': ['Quel est ton but ',
    'Quel est ton travail',
    'A quoi tu sers ? '],
   'responses': ['Je suis un assistant qui permet de vous rediriger au mieux sur le site web',
    'Je suis là pour rendre votre vie meilleure'],
   'tag': 'job'},
  {'context': '',
   'patterns': ['Quel âge as tu', "T'as quel age", 'Quand tu es né'],
   'responses': ['Je suis né en 2023'],
   'tag': 'age'},
  {'context': '',
   'patterns': ["Comment ça va aujourd'hui", 'Comment ça va ?'],
   'responses': ['Je vais bien et vous?',
    'Très bien et vous?',
    'Ca va plutôt bien et vous?'],
   '

In [51]:
#4 Extracting data_X(features) and data_Y(Target)

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow  model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X 
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # tokenize each pattern 
        words.extend(tokens) #and append tokens to words
        data_X.append(pattern) #appending pattern to data_X
        data_y.append(intent["tag"]) ,# appending the associated tag to each pattern 
    
    # adding the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [52]:
# 5 Text to Numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [53]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               10752     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 17)                1105      
                                                                 
Total params: 20,113
Trainable params: 20,113
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
2/2 [============================

In [54]:
#7 Preprocessing the Input

def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability 
  return return_list

def get_response(intents_list, intents_json): 
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [55]:
model.save('chatbotmodel')

In [56]:
import pickle as pkl
import joblib 

In [57]:
joblib.dump(model, 'model.pkl')

['model.pkl']

In [58]:
# Interacting with the chatbot
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
    message = input("")
    if message == "0":
      break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Press 0 if you don't want to chat with our ChatBot.
Bonjour
1/1 [==============================] - 0s 92ms/step
Sorry! I don't understand.
bonjour
1/1 [==============================] - 0s 25ms/step
Comment-allez vous
je vais bien 
1/1 [==============================] - 0s 22ms/step
That is perfect!
t'habites ou 
1/1 [==============================] - 0s 22ms/step
Hey!
tu vis ou 
1/1 [==============================] - 0s 21ms/step
Je vis dans un serveur!
tu fais quoi 
1/1 [==============================] - 0s 39ms/step
Je suis actuellement en train de chatter avec vous
ok 
1/1 [==============================] - 0s 26ms/step
Sorry! I don't understand.
pas de soucis, tu peux patienter ?
1/1 [==============================] - 0s 31ms/step
Je peux faire presque tout ce que vous voulez !
la formation est payante ? 
1/1 [==============================] - 0s 22ms/step
Non, la formation n'est pas payante
la formation dure combien de temps ? 
1/1 [==============================] - 0s 25ms/step


KeyboardInterrupt: ignored

In [59]:
!pip freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
astropy==5.2.2
astunparse==1.6.3
attrs==23.1.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blinker==1.6.2
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
distributed==2022.12.